In [22]:
import pandas as pd
from pandas import DataFrame, Series
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import random

import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from datetime import date
from dateutil.relativedelta import relativedelta

In [23]:
TRIP_FILE = ('201508_trip_data.csv')
#WEATHER = ('201508_weather_data.csv')
#STATION_BALANCE = ('201508_status_data.csv')
STATION_DATA = ('201508_station_data.csv')

trip_data = pd.read_csv(TRIP_FILE)
#weather_data = pd.read_csv(WEATHER)
#station_balance_data = pd.read_csv(STATION_BALANCE)
station_data = pd.read_csv(STATION_DATA)

trip_data.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139
1,913459,1036,8/31/2015 23:11,San Antonio Shopping Center,31,8/31/2015 23:28,Mountain View City Hall,27,35,Subscriber,95032
2,913455,307,8/31/2015 23:13,Post at Kearny,47,8/31/2015 23:18,2nd at South Park,64,468,Subscriber,94107
3,913454,409,8/31/2015 23:10,San Jose City Hall,10,8/31/2015 23:17,San Salvador at 1st,8,68,Subscriber,95113
4,913453,789,8/31/2015 23:09,Embarcadero at Folsom,51,8/31/2015 23:22,Embarcadero at Sansome,60,487,Customer,9069


In [24]:
station_data_sf = station_data[station_data.landmark == 'San Francisco'].copy()
station_data_sf.head()

,station_id,name,lat,long,dockcount,landmark,installation
32,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,8/19/2013
33,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,8/19/2013
34,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,8/19/2013
35,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,8/19/2013
36,47,Post at Kearney,37.788975,-122.403452,19,San Francisco,8/19/2013


In [25]:
trip_data = trip_data.merge(station_data[['station_id', 'lat', 'long', 'landmark']], left_on = 'Start Terminal', right_on = 'station_id')
#trip_data.join(station_data, on='station_id')
trip_data = trip_data.merge(station_data[['station_id', 'lat', 'long']], left_on = 'End Terminal', right_on = 'station_id')

trip_data.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,station_id_x,lat_x,long_x,landmark,station_id_y,lat_y,long_y
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
1,913228,886,8/31/2015 18:47,Harry Bridges Plaza (Ferry Building),50,8/31/2015 19:02,San Francisco Caltrain (Townsend at 4th),70,387,Subscriber,94107,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
2,913009,725,8/31/2015 17:40,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:53,San Francisco Caltrain (Townsend at 4th),70,360,Subscriber,94111,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
3,912956,1034,8/31/2015 17:31,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:48,San Francisco Caltrain (Townsend at 4th),70,587,Subscriber,94087,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
4,912776,871,8/31/2015 16:50,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:04,San Francisco Caltrain (Townsend at 4th),70,371,Subscriber,94070,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526


In [26]:
trip_data_sf = trip_data[trip_data.landmark == 'San Francisco'].copy()
trip_data_sf.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,station_id_x,lat_x,long_x,landmark,station_id_y,lat_y,long_y
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
1,913228,886,8/31/2015 18:47,Harry Bridges Plaza (Ferry Building),50,8/31/2015 19:02,San Francisco Caltrain (Townsend at 4th),70,387,Subscriber,94107,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
2,913009,725,8/31/2015 17:40,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:53,San Francisco Caltrain (Townsend at 4th),70,360,Subscriber,94111,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
3,912956,1034,8/31/2015 17:31,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:48,San Francisco Caltrain (Townsend at 4th),70,587,Subscriber,94087,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
4,912776,871,8/31/2015 16:50,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:04,San Francisco Caltrain (Townsend at 4th),70,371,Subscriber,94070,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526


In [27]:
trip_data_sf.rename(columns={'station_id_x': 'start_station', 'lat_x': 'start_lat', 'long_x': 'start_long', 'station_id_y': 'end_station', 'lat_y': 'end_lat', 'long_y': 'end_long'}, inplace=True)
#trip_data_sf.drop('name', axis=1, inplace=True)
trip_data_sf.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscriber Type,Zip Code,start_station,start_lat,start_long,landmark,end_station,end_lat,end_long
0,913460,765,8/31/2015 23:26,Harry Bridges Plaza (Ferry Building),50,8/31/2015 23:39,San Francisco Caltrain (Townsend at 4th),70,288,Subscriber,2139,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
1,913228,886,8/31/2015 18:47,Harry Bridges Plaza (Ferry Building),50,8/31/2015 19:02,San Francisco Caltrain (Townsend at 4th),70,387,Subscriber,94107,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
2,913009,725,8/31/2015 17:40,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:53,San Francisco Caltrain (Townsend at 4th),70,360,Subscriber,94111,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
3,912956,1034,8/31/2015 17:31,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:48,San Francisco Caltrain (Townsend at 4th),70,587,Subscriber,94087,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526
4,912776,871,8/31/2015 16:50,Harry Bridges Plaza (Ferry Building),50,8/31/2015 17:04,San Francisco Caltrain (Townsend at 4th),70,371,Subscriber,94070,50,37.795392,-122.394203,San Francisco,70,37.776617,-122.39526


In [28]:
list(trip_data_sf.columns.values)

['Trip ID',
 'Duration',
 'Start Date',
 'Start Station',
 'Start Terminal',
 'End Date',
 'End Station',
 'End Terminal',
 'Bike #',
 'Subscriber Type',
 'Zip Code',
 'start_station',
 'start_lat',
 'start_long',
 'landmark',
 'end_station',
 'end_lat',
 'end_long']

In [29]:
import scipy
from scipy.spatial import distance

In [30]:
origin = trip_data_sf.as_matrix(['start_lat', 'start_long'])
dest = trip_data_sf.as_matrix(['end_lat', 'end_long'])
print('Checking array shapes...')
print(origin)
print(dest)

Checking array shapes...
[[  37.795392 -122.394203]
 [  37.795392 -122.394203]
 [  37.795392 -122.394203]
 ..., 
 [  37.783871 -122.408433]
 [  37.792251 -122.397086]
 [  37.786305 -122.404966]]
[[  37.776617 -122.39526 ]
 [  37.776617 -122.39526 ]
 [  37.776617 -122.39526 ]
 ..., 
 [  37.400443 -122.108338]
 [  37.491269 -122.236234]
 [  37.48537  -122.203288]]


In [15]:
trip_sample = trip_data_sf.sample(frac=.01)
print(trip_sample)
print(type(trip_sample))

        Trip ID  Duration        Start Date  \
238071   536626      2481   11/9/2014 13:35   
199531   671838       802    3/6/2015 20:27   
235458   464598       727   9/22/2014 18:14   
216755   650224       363   2/19/2015 13:53   
255551   712078       374    4/4/2015 11:07   
202818   705635       322   3/31/2015 10:16   
163725   445467       245    9/9/2014 20:37   
35886    561222       171  11/30/2014 12:52   
102715   904474       657   8/25/2015 14:23   
248696   829123      1027   6/30/2015 19:13   
158230   592781       561    1/6/2015 13:12   
41633    888057       472    8/13/2015 8:48   
250273   778897        93   5/22/2015 20:42   
54146    594742       648    1/7/2015 16:32   
79983    573844       939  12/10/2014 17:56   
313204   696776       471    3/25/2015 8:55   
254309   800539       246    6/9/2015 17:30   
267837   793345       691     6/4/2015 7:46   
253435   660382      1475   2/26/2015 20:43   
4977     796378       336    6/5/2015 18:09   
209262   8448

In [17]:
trip_sample.to_csv("trip_sample.csv")

In [31]:
def measure_distance(start_lat, start_long, end_lat, end_long):
    x_difference = start_lat - end_lat
    y_difference = start_long - end_long
    x_squared = x_difference**2
    y_squared = y_difference**2
    added = x_squared + y_squared
    squart_root_ = np.sqrt(added)
    to_feet = squart_root_ * 2390.31 / 0.0082459784137541101
    return to_feet

In [36]:
def compute_distance_df(trip_data_sf, start_lat_name, start_long_name, end_lat_name, end_long_name):
    distances_list = []
    for row in range(len(trip_data_sf)):
        distance_between_points = measure_distance(trip_data_sf[start_lat_name][row], trip_data_sf[start_long_name][row],\
                                trip_data_sf[end_lat_name][row], trip_data_sf[end_long_name][row])
        distances_list.append(distance_between_points)
    return distances_list

In [37]:
distances_list = compute_distance_df(trip_data_sf,"start_lat","start_long","end_lat","end_long")

KeyError: 14628

In [5]:
#Converting 'str' objects to 'int'in col='Duration'
#list(map(int, trip_data['Duration']))

In [32]:
#Converting to datetime
#trip_data['Start Date'] = pd.to_datetime(trip_data['Start Date'])
#trip_data['End Date'] = pd.to_datetime(trip_data['End Date'])

In [35]:
#station_balance_data.time = pd.to_datetime(station_balance_data.time)

In [36]:
type(trip_data_sf[['start_lat', 'start_long']])

pandas.core.frame.DataFrame

In [38]:
trip_data_sf.as_matrix(['start_lat', 'start_long'])

array([[  37.795392, -122.394203],
       [  37.795392, -122.394203],
       [  37.795392, -122.394203],
       ..., 
       [  37.783871, -122.408433],
       [  37.792251, -122.397086],
       [  37.786305, -122.404966]])